In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Librarires to collect stock price data.

- pip install pandas_datareader
- pip install yfinance

In [ ]:
yf.pdr_override()

# download dataframe
df = pdr.get_data_yahoo("AAPL", start="2020-11-01", end="2022-09-01", interval='60m')

df

In [ ]:
data = df.iloc[:, 0: 1].values

In [ ]:
data

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
data = sc.fit_transform(data)

In [ ]:
X = []
y  = []

for i in range(60, len(data)):
    X.append(data[i-60: i, 0])
    y.append(data[i, 0])
    
X, y = np.array(X), np.array(y)

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .02, random_state = 42)

In [ ]:
print(X_train.shape)
X_train = np.reshape(X_train, newshape = (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, newshape = (X_test.shape[0], X_test.shape[1], 1))
X_test

### RNN

In [ ]:
rnn = keras.Sequential()

rnn.add(keras.layers.SimpleRNN(128, return_sequences=True))
rnn.add(keras.layers.SimpleRNN(128, return_sequences=True))
rnn.add(keras.layers.SimpleRNN(256, return_sequences=True))
rnn.add(keras.layers.SimpleRNN(32))
rnn.add(keras.layers.Dense(units = 1))

rnn.compile(optimizer='rmsprop', loss='mean_squared_error')
rnn.fit(X_train, y_train, epochs=100, batch_size=32)

### LSTM

- Dropout: neurons/ data intentionally dropped from a neural network to improve processing and time to results
- Finally later --> Dense (units = number of predictions)

In [ ]:
lstm = keras.Sequential()

lstm.add(keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
lstm.add(keras.layers.Dropout(rate = 0.2))
lstm.add(keras.layers.LSTM(units = 50, return_sequences = True))
lstm.add(keras.layers.Dropout(rate = 0.2))
lstm.add(keras.layers.LSTM(units = 50, return_sequences = True))
lstm.add(keras.layers.Dropout(rate = 0.2))
lstm.add(keras.layers.LSTM(units = 50, return_sequences = False))
lstm.add(keras.layers.Dropout(rate = 0.2))
lstm.add(keras.layers.Dense(units = 1))

lstm.compile(optimizer = 'adam', loss = 'mean_squared_error')
lstm.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100)

### GRU

In [ ]:
gru = keras.Sequential()

gru.add(keras.layers.GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],1), activation='tanh'))
gru.add(keras.layers.GRU(units=50, return_sequences=True))
gru.add(keras.layers.GRU(units=50, return_sequences=True))
gru.add(keras.layers.GRU(units=50, return_sequences=False, activation='tanh'))
gru.add(keras.layers.Dense(units=1))

history = gru.compile(optimizer='rmsprop', loss='mean_squared_error')
gru.fit(X_train, y_train, batch_size = 32, epochs = 100)

In [ ]:
predicted_rnn = rnn.predict(X_test)
predicted_lstm = lstm.predict(X_test)
predicted_gru = gru.predict(X_test)

predicted_rnn = sc.inverse_transform(predicted_rnn)
predicted_lstm = sc.inverse_transform(predicted_lstm)
predicted_gru = sc.inverse_transform(predicted_gru)

In [ ]:
org = sc.inverse_transform(y_test.reshape(-1, 1))

plt.plot(org, label = 'Real price')
plt.plot(predicted_rnn, label = 'Recurrent Neural Network')
plt.plot(predicted_lstm, label = 'Long Short Term Memory Network')
plt.plot(predicted_gru, label = 'Gated Recurrent Unit')
plt.title('Google price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
rmse_rnn = np.sqrt(mean_squared_error(org, predicted_rnn))
rmse_lstm = np.sqrt(mean_squared_error(org, predicted_lstm))
rmse_gru = np.sqrt(mean_squared_error(org, predicted_gru))

print ('Test RMSE - RNN: %.3f' % rmse_rnn)
print ('Test RMSE - LSTM: %.3f' % rmse_lstm)
print ('Test RMSE - GRU: %.3f' % rmse_gru)